In [1]:
# Import necessary libraries
import os
from tensorflow import keras
import numpy as np
import tensorflow as tf
from keras import layers, models
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from lxml import etree

In [2]:
# Define paths to annotation and image directories
annotations_dir = "C:\\Users\\mazen\\Desktop\\archive\\imagess\\img-annotations"
images_dir = "C:\\Users\\mazen\\Desktop\\archive\\imagess\\image"

# Define classes
classes = ['With Helmet', 'Without Helmet']

In [ ]:
# Parse XML annotations
def parse_annotation(annotation_file):
    tree = etree.parse(annotation_file)
    root = tree.getroot()
    objects = []
    for obj in root.findall('object'):
        obj_struct = {}
        obj_struct['name'] = obj.find('name').text
        bbox = obj.find('bndbox')
        obj_struct['bbox'] = [int(bbox.find('xmin').text), int(bbox.find('ymin').text),
                              int(bbox.find('xmax').text), int(bbox.find('ymax').text)]
        objects.append(obj_struct)
    return objects

